In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import os
import glob
from nilearn.image import resample_img
from nilearn import plotting
from nilearn.image import mean_img
from nilearn.input_data import NiftiMasker
import matplotlib.pyplot as plt
import statistics 
from datetime import date
import math
from nilearn.image import concat_imgs, mean_img
from nilearn.decoding import Decoder
from nilearn.masking import apply_mask

#python3 /Volumes/GoogleDrive/My\ Drive/U01/working_memory/analysis/MVPA/1vs3back/1vs3back.py

220709-10:57:39,280 nipype.utils WARNING:
	 A newer version (1.8.1) of nipy/nipype is available. You are using 1.7.0


/Users/chendanlei/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [3]:
clfLabel1 = ['3-back']
clfLabel2 = ['1-back']
base_dir = '/Volumes/GoogleDrive/My Drive/U01/working_memory/analysis/univariate/results/level1/wm_nback_block/SC_warp/'
output_dir='/Volumes/GoogleDrive/My Drive/U01/working_memory/analysis/MVPA/1vs3back/'
mask_filename= '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/avg_template/Template_6_thresh0.25_resammpled_bin.nii.gz'
# mask_filename= '/Users/chendanlei/Desktop/Template_6_thresh0.25_resammpled_bin.nii.gz'
cope_files1 = [i for i in glob.glob(base_dir+'sub*/'+'*cope*.nii.gz') if i.split('_cope')[-1].split('.nii')[0] in ['1','2','3','4','5','6']]
cope_files2 = [i for i in glob.glob(base_dir+'sub*/'+'*cope*.nii.gz') if i.split('_cope')[-1].split('.nii')[0] in ['7','8','9','10','11','12']]
cope_files1.sort()
cope_files2.sort()
all_files = cope_files1+cope_files2
######
fold_num_list=[88,12,6]
output_df= pd.DataFrame(columns = [str(i)+' folds' for i in fold_num_list])#two columns for k fold and two for leave one out

output_event_name = output_dir+'1vs3back_event'
output_name = output_dir+'1vs3back_MVPA_classification'


In [4]:
mask_data = resample_img(mask_filename,
                    target_affine=nib.load(cope_files1[0]).affine,
                    target_shape=nib.load(cope_files1[0]).shape[0:3],
                    interpolation='nearest').get_fdata()
mask_data[mask_data!=1]=0
mask_img = nib.Nifti1Image(mask_data, nib.load(cope_files1[0]).affine, nib.load(cope_files1[0]).header)


/Users/chendanlei/.local/lib/python3.8/site-packages/nilearn/image/resampling.py:598: RuntimeWarning: NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
  _resample_one_img(data[all_img + ind], A, b, target_shape,


In [18]:
# #concatenate all files
# # concat_img = concat_imgs(all_files)
# # nib.save(concat_img, output_dir+'concatenated_img.nii.gz')
# concat_img = nib.load(output_dir+'concatenated_img.nii.gz')
# #apply mask on combined image
# masker = NiftiMasker(mask_img=mask_img, standardize=False)
# concat_data2D_masked = masker.fit_transform(concat_img)
# np.save(output_dir+'concatenated_img_SCmasked.npy', concat_data2D_masked)
# # concat_img_masked = nib.load(output_dir+'concatenated_img_SCmasked.nii.gz')
# # concat_data_masked = concat_img_masked.get_fdata()
# concat_img_masked=masker.inverse_transform(concat_data2D_masked)
# nib.save(concat_img_masked, output_dir+'concatenated_img_SCmasked.nii.gz')
# nib.save(nib.Nifti1Image(concat_img_masked, nib.load(cope_files1[0]).affine, nib.load(cope_files1[0]).header), output_dir+'concatenated_img_SCmasked.nii.gz')
# concat_img_masked = nib.load(output_dir+'concatenated_img_SCmasked.nii.gz')
# concat_data2D_masked = np.load(output_dir+'concatenated_img_SCmasked.npy')
# print('masked combined image shape:')
# print(concat_img_masked.shape)


masked combined image shape:
(176, 208, 176, 1044)


In [8]:
# np.load(output_dir+'concatenated_img_SCmasked.npy')


In [9]:
# event_files = pd.DataFrame(np.array([np.concatenate([np.repeat(clfLabel1,len(cope_files1)), np.repeat(clfLabel2,len(cope_files2))]), 
#                            ['block'+str(int(i.split('_cope')[-1].split('.nii')[0])%6) for i in all_files], 
#                            [i.split('/')[-1].split('wdata_')[-1].split('__')[0] for i in all_files]]).T, 
#                          columns=['labels','block', 'subject'])
event_files = pd.DataFrame(np.array([np.concatenate([np.repeat(clfLabel1,len(cope_files1)), np.repeat(clfLabel2,len(cope_files2))]), 
                           [i.split('/')[-1].split('w_')[-1].split('__')[0] for i in all_files]]).T, 
                         columns=['labels','subject'])

print('event file shape:')
print(event_files.shape)
condition = event_files['labels']
session = event_files['subject']
event_files.to_csv((output_event_name+'.tsv'),index=True,sep='\t')


event file shape:
(1044, 2)


In [34]:
#########different methods of prediction########
# k-fold loop
from sklearn.svm import SVC
svc = SVC(kernel='linear')

#similar k-fold method using packages nilearn decoder 
print('k-fold running')
from sklearn.model_selection import KFold
for fold_num in fold_num_list:
    print(fold_num)
    cv = KFold(n_splits=fold_num)
#     decoder = Decoder(estimator='svc', standardize=True,
#                   screening_percentile=5, scoring='accuracy', cv=cv)
#     decoder.fit(output_dir+'concatenated_img_SCmasked.nii.gz', np.array(condition), groups=np.array(session))
    decoder = Decoder(estimator='svc', mask=mask_img, standardize=True,
                  screening_percentile=5, scoring='accuracy', cv=cv)
    decoder.fit(output_dir+'concatenated_img.nii.gz', condition, groups=session)

    print(decoder.cv_scores_[clfLabel1[0]])
    print(decoder.cv_scores_[clfLabel2[0]])
    # #dummy classifier to compare with
    # dummy_classifier = DummyClassifier()
    # dummy_scores = cross_val_score(dummy_classifier,concat_img_masked, condition, cv=cv,groups=session, scoring="accuracy",)
    
    output_df= pd.DataFrame(columns = [str(fold_num)+' folds'])#two columns for k fold and two for leave one out
    output_df[str(fold_num)+' folds'] = decoder.cv_scores_[clfLabel1[0]]
    output_df.to_csv((output_name+'_'+str(fold_num)+'folds.tsv'),index=True,sep='\t')
    
    weight_img = decoder.coef_img_[clfLabel1[0]]
    nib.save(weight_img, output_dir+clfLabel1[0]+'_weights_'+str(fold_num)+'folds.nii.gz')
    weight_img = decoder.coef_img_[clfLabel2[0]]
    nib.save(weight_img, output_dir+clfLabel2[0]+'_weights_'+str(fold_num)+'folds.nii.gz')
    # from nilearn.plotting import plot_stat_map, show
    # plot_stat_map(weight_img, title='SVM weights')
    # show()
    

k-fold running
88


/Users/chendanlei/.local/lib/python3.8/site-packages/nilearn/_utils/param_validation.py:197: UserWarning: Brain mask is smaller than .5% of the volume human brain. This object is probably not tuned tobe used on such data.
  screening_percentile_ = _adjust_screening_percentile(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarnin

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

...

[0.5, 0.6666666666666666, 0.25, 0.5, 0.8333333333333334, 0.5833333333333334, 0.5, 0.4166666666666667, 0.5833333333333334, 0.4166666666666667, 0.5, 0.6666666666666666, 0.6666666666666666, 0.4166666666666667, 0.5, 0.5833333333333334, 0.6666666666666666, 0.4166666666666667, 0.6666666666666666, 0.8333333333333334, 0.5, 0.4166666666666667, 0.5, 0.5833333333333334, 0.4166666666666667, 0.5833333333333334, 0.6666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.6666666666666666, 0.4166666666666667, 0.4166666666666667, 0.6666666666666666, 0.5, 0.5, 0.5833333333333334, 0.25, 0.6666666666666666, 0.3333333333333333, 0.5833333333333334, 0.3333333333333333, 0.4166666666666667, 0.5, 0.4166666666666667, 0.5833333333333334, 0.6666666666666666, 0.4166666666666667, 0.5, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.5, 0.5833333333333334, 0.3333333333333333, 0.25, 0.6666666666666666, 0.6666666666666666, 0.75, 0.8333333333333334, 0.3333333333333333, 0.6666666666666666, 0.41666666

/Users/chendanlei/.local/lib/python3.8/site-packages/nilearn/_utils/param_validation.py:197: UserWarning: Brain mask is smaller than .5% of the volume human brain. This object is probably not tuned tobe used on such data.
  screening_percentile_ = _adjust_screening_percentile(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarnin

[0.45977011494252873, 0.4482758620689655, 0.5172413793103449, 0.45977011494252873, 0.40229885057471265, 0.41379310344827586, 0.4942528735632184, 0.47126436781609193, 0.4482758620689655, 0.5747126436781609, 0.4482758620689655, 0.5977011494252874]
[0.45977011494252873, 0.4482758620689655, 0.5172413793103449, 0.45977011494252873, 0.40229885057471265, 0.41379310344827586, 0.4942528735632184, 0.47126436781609193, 0.4482758620689655, 0.5747126436781609, 0.4482758620689655, 0.5977011494252874]
6


/Users/chendanlei/.local/lib/python3.8/site-packages/nilearn/_utils/param_validation.py:197: UserWarning: Brain mask is smaller than .5% of the volume human brain. This object is probably not tuned tobe used on such data.
  screening_percentile_ = _adjust_screening_percentile(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarnin

[0.3735632183908046, 0.4540229885057471, 0.3160919540229885, 0.42528735632183906, 0.45977011494252873, 0.5172413793103449]
[0.3735632183908046, 0.4540229885057471, 0.3160919540229885, 0.42528735632183906, 0.45977011494252873, 0.5172413793103449]


In [17]:
# from sklearn.model_selection import KFold
# cv = KFold(n_splits=fold_num)
# # The "cv" object's split method can now accept data and create a
# # generator which can yield the splits.
# fold_predict = []
# for train, test in cv.split(X=concat_data_masked):
#     condition_masked = condition.values[train]
#     session_masked = session.values[train]
#     svc.fit(concat_data_masked[train], condition_masked)
#     prediction = svc.predict(concat_data_masked[test])
#     fold_predict.append((prediction == condition.values[test]).sum()/ float(len(condition.values[test])))
# print('*************************')
# print(str(fold_num)+' folds loop prediction')
# print(fold_predict)
# print('mean: '+ str(statistics.mean(fold_predict)))
# print('*************************')
# file.write('\n') 
# if statistics.mean(fold_predict)>0.5:
#     file.write('***********************************')
# file.write(str(fold_num)+' folds loop prediction'+'\n') 
# file.writelines("%s " % i for i in fold_predict)
# file.write('\nMEAN: '+ str(statistics.mean(fold_predict))+'\n')
# file.write('SE: '+ str(statistics.stdev(fold_predict)/math.sqrt(max(session_masked)))+'\n')

# #similar k-fold method using packages skilearn 
# print('k-fold running')
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# for fold_num in fold_num_list:
#     cv = KFold(n_splits=fold_num)
#     cv_score = cross_val_score(svc, concat_data2D_masked, condition, cv=cv, groups=session)
#     # print('*************************')
#     # print(str(fold_num)+' folds prediction:')
#     # print(cv_score)
#     # print('mean: '+ str(statistics.mean(cv_score)))
#     # print('*************************')
#     k_mean = statistics.mean(cv_score)
#     k_se = statistics.stdev(cv_score)/math.sqrt(len(np.unique(session)))


k-fold running


NameError: name 'concat_data2D_masked' is not defined

In [ ]:
c